In [1]:
import urllib
import warnings
from pathlib import Path as p
from pprint import pprint

import pandas as pd
from langchain import PromptTemplate
from langchain.chains.question_answering import load_qa_chain
from langchain.document_loaders import PyPDFLoader
from langchain.embeddings import VertexAIEmbeddings
from langchain.llms import VertexAI
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Chroma
from IPython.display import display, Markdown

warnings.filterwarnings("ignore")

In [2]:
# from langchain.document_loaders import PyPDFLoader
# from langchain.document_loaders import CSVLoader
# file = 'OutdoorClothingCatalog_1000.csv'
# loader1 = CSVLoader(file_path=file)
# pdfFile = '/OfferCreationviaGrumbles.pdf'

# loader1

In [3]:
import pathlib
desktop = pathlib.Path("./SOP")
files = []
list(desktop.iterdir())
pages1 = []
# for item in list(desktop.iterdir())[:1]:
for item in list(desktop.iterdir()):
    try:
        print(str(item))
        fileLoader = PyPDFLoader(str(item))
#         pages = fileLoader.load_and_split()
#         print(len(fileLoader.load_and_split()))
        pages1.append(fileLoader.load_and_split())
#         files.append(pages)
    except:
        print("An exception occurred")
  
print (type(pages1))


SOP/Incentivised Attach SOP.pdf
SOP/Targeted Discounts - SOP.pdf
SOP/Bump Up Offers.pdf
SOP/Casper SOP.pdf
SOP/Targeted Discount on FP - SOP.pdf
SOP/_SOP for Shipping Rule Creation.pdf
SOP/Exchange Offers.pdf
SOP/Prebook Offer.pdf
SOP/Shipping Offers - SOP.pdf
SOP/Either Or offers SOP.pdf
SOP/Promise Callout Offers - SOP.pdf
SOP/BMGN v1 Offer SOP.pdf
SOP/Callout Offer.pdf
SOP/CnC Offers - SOP.pdf
SOP/Multi Unit Listing Offers - SOP.pdf
SOP/Seller Recommendation for an Offer.pdf
SOP/Offer Creation via Grumbles .pdf
SOP/Basket Offers - SOP (1).pdf
SOP/Early Access SOP.pdf
SOP/Combo Offers - SOP.pdf
SOP/PBO SOP.pdf
<class 'list'>


In [4]:


pages = []
print(len(pages1))
for item in pages1:
    try:
        if(item.page_content):
            pages.append(item)
            print(item)
    except:
        for item2 in item:
            pages.append(item2)
print(len(pages))

context = "\n".join(str(p.page_content) for p in pages)
print(type(context))
print(len(context))

21
207
<class 'str'>
228226


In [7]:
from langchain.vectorstores import DocArrayInMemorySearch
db = DocArrayInMemorySearch.from_documents(
    pages, 
    vertex_embeddings
)

In [6]:
vertex_llm_text = VertexAI(model_name="text-bison@001")
vertex_embeddings = VertexAIEmbeddings(model_name="textembedding-gecko@001")

In [9]:
# query = "how does deferred Giveaway works for offer"
# query = "how does I create a Promise Callout Offer"
query = "What is Insufficient Participation?"
relevantDocs = db.similarity_search(query, k=5)
# relevantDocs = db.similarity_search_with_score(query, k=10)

print(len(relevantDocs))
# print(relevantDocs)

qdocs = "".join([relevantDocs[i].page_content for i in range(len(relevantDocs))])
# print(qdocs)
context = qdocs
for i in range(0,len(qdocs)-2):
    print(relevantDocs[i])
    relevantDocs[i].page_content
    print(item, "======\n")

5
page_content='PBO\nOverview\nand\nSOP\n1 2 .\nU p d a t e\nr e q u i r e d\nd e t a i l s\ni n\nP a y m e n t\nc a t e g o r y\n(\nl i k e\np a y m e n t\ni n s t r u m e n t\na n d\nE M I\nt y p e\ni f\na p p l i c a b l e )\n1 3 .\nE n t e r\nF K\na d v a n z\no r\ns e l l e r\nb u r n\na s\n1 0 0 %\nb a s e d\no n\nr e q u i r e m e n t .\n1 4 .\nS e l e c t\np r i m a r y\no b j e c t i v e\na s\nG M V\nm a x i m i z a t i o n\na n d\ne x p e c t e d\nR P C\nd e f a u l t\nv a l u e\na s\n1 .\nC l i c k\no n\nc o n t i n u e\nf e a t u r e .' metadata={'source': 'SOP/PBO SOP.pdf', 'page': 37}
[Document(page_content='PBO\nOverview\nand\nSOP\nI m p o r t a n t\nT e r m s\na n d\nC o n c e p t s\n2\nC r e a t i n g\nP B O s\n3\nS i t e w i d e\n:\n1 0 0 %\nF A P L\nb u r n\no f f e r\n3\nC r e a t i n g\nN C\nE M I\nO f f e r s\n1 4\n1 0 0 %\nF K\nb u r n\no f f e r ( B F L\no f f e r ) :\nA p p l i c a b l e\no n\nc e r t a i n\nc a t e g o r i e s\n( M o b i l e ,\nE l e , L a r g

IndexError: list index out of range

In [10]:
question = "What is Insufficient Participation?"
question_prompt_template = """
                    Answer the question as precise as possible using the provided context. \n\n
                    Context: \n {context} \n
                    Question: \n {question} \n
                    Answer:
                    """
# question_prompt = PromptTemplate(
#     template=question_prompt_template, input_variables=["context", "question"]
# )

question_prompt = PromptTemplate(
    template=question_prompt_template, 
    input_variables=["question", "context"]
)

# summaries is required. a bit confusing.
combine_prompt_template = """Given the extracted content and the question, create a final answer.
If the answer is not contained in the context, say "answer not available in context. \n\n
Summaries: \n {summaries}?\n
Question: \n {question} \n
Answer:
"""


In [11]:
combine_prompt = PromptTemplate(
    template=combine_prompt_template, input_variables=[ "summaries","question"]
)


In [12]:
map_reduce_chain = load_qa_chain(
    vertex_llm_text,
    chain_type="map_reduce",
    return_intermediate_steps=True,
    question_prompt=question_prompt,
    combine_prompt=combine_prompt,
)


In [16]:
import langchain
langchain.debug = True
map_reduce_outputs = map_reduce_chain({"input_documents": pages, "question": question})

[chain/start] [1:chain:MapReduceDocumentsChain] Entering Chain run with input:
[inputs]
[chain/start] [1:chain:MapReduceDocumentsChain > 2:chain:LLMChain] Entering Chain run with input:
{
  "input_list": [
    {
      "context": "W h a t\na r e\nI n c e n t i v i s e d\nA t t a c h\nO \ufb00 e r s ?\nB u y\nP 1 ,\ng e t\nx %\no \ufb00\no n\nP 2\nI n c e n t i v i s e d\nA t t a c h\nO \ufb00 e r s\na r e\nm u l t i\np r o d u c t\ng r o u p\no \ufb00 e r s\nt h a t\nh a v e\na\nh u r d l e\no n\n1\np r o d u c t\ng r o u p ,\na n d\na\ng i v e a w a y\no n\na\nd i \ufb00 e r e n t\np r o d u c t\ng r o u p .\nP r o d u c t\ng r o u p\nr e f e r s\nt o\na\ns e t\no f\nF S N s\nd e \ufb01 n e d\nb y\nt h e\no \ufb00 e r\nc r e a t o r .\nI f\nt h e\nc o m b o\no \ufb00 e r\nh a s\n2\nP G s ,\na\nu s e r\nn e e d s\nt o\nb u y\na n\ni t e m\nf r o m\ne a c h\no f\nt h e\nP G\nw h i c h\nw i l l\nm a k e\nh i m / h e r\ne l i g i b l e\nf o r\nd i s c o u n t\no n\nt h e\ns e c o n d\nP G 

[chain/error] [1:chain:MapReduceDocumentsChain > 2:chain:LLMChain > 3:llm:VertexAI] [116.42s] Chain run errored with error:
"ResourceExhausted('Quota exceeded for aiplatform.googleapis.com/online_prediction_requests_per_base_model with base model: text-bison. Please submit a quota increase request. https://cloud.google.com/vertex-ai/docs/quotas.')"
[chain/error] [1:chain:MapReduceDocumentsChain > 2:chain:LLMChain] [116.43s] Chain run errored with error:
"ResourceExhausted('Quota exceeded for aiplatform.googleapis.com/online_prediction_requests_per_base_model with base model: text-bison. Please submit a quota increase request. https://cloud.google.com/vertex-ai/docs/quotas.')"
[chain/error] [1:chain:MapReduceDocumentsChain] [116.44s] Chain run errored with error:
"ResourceExhausted('Quota exceeded for aiplatform.googleapis.com/online_prediction_requests_per_base_model with base model: text-bison. Please submit a quota increase request. https://cloud.google.com/vertex-ai/docs/quotas.')"


ResourceExhausted: 429 Quota exceeded for aiplatform.googleapis.com/online_prediction_requests_per_base_model with base model: text-bison. Please submit a quota increase request. https://cloud.google.com/vertex-ai/docs/quotas.

In [ ]:
# !pip install transformers

In [ ]:
final_mp_data = []

# for each document, extract metadata and intermediate steps of the MapReduce process
for doc, out in zip(
    map_reduce_outputs["input_documents"], map_reduce_outputs["intermediate_steps"]
):
    output = {}
    output["file_name"] = p(doc.metadata["source"]).stem
    output["file_type"] = p(doc.metadata["source"]).suffix
    output["page_number"] = doc.metadata["page"]
    output["chunks"] = doc.page_content
    output["answer"] = out
    final_mp_data.append(output)

In [ ]:
# create a dataframe from a dictionary
pdf_mp_answers = pd.DataFrame.from_dict(final_mp_data)
# sorting the dataframe by filename and page_number
pdf_mp_answers = pdf_mp_answers.sort_values(by=["file_name", "page_number"])
pdf_mp_answers.reset_index(inplace=True, drop=True)
pdf_mp_answers.head()
print(pdf_mp_answers)

In [ ]:
# !pip install docarray
from langchain.vectorstores import DocArrayInMemorySearch
db = DocArrayInMemorySearch.from_documents(
    pages, 
    vertex_embeddings
)

In [ ]:
# print(type(pages))

In [ ]:
### we will use above vectorstore to find answer to query
# query = "What is Incentivised offer"

In [ ]:
## find all similar document
# docs = db.similarity_search(query)
# print(len(docs))
## 4 docs are returned
# docs[0] ## indeed a shirt

In [ ]:
retriever = db.as_retriever()

In [ ]:
from vertexai.preview.language_models import TextGenerationModel


def getResponse(prompt ,temperature: float = 0.2):
    """Ideation example with a Large Language Model"""

    # TODO developer - override these parameters as needed:
    parameters = {
        "temperature": temperature,  # Temperature controls the degree of randomness in token selection.
        "max_output_tokens": 256,  # Token limit determines the maximum amount of text output.
        "top_p": 0.8,  # Tokens are selected from most probable to least until the sum of their probabilities equals the top_p value.
        "top_k": 40,  # A top_k of 1 means the selected token is the most probable among all tokens.
    }

    model = TextGenerationModel.from_pretrained("text-bison@001")
    response = model.predict(
        prompt,
        **parameters,
    )
    
    return response


In [ ]:

# from vertexai.preview.language_models import ChatModel, InputOutputTextPair

# def getChatResponse(prompt, temperature: float = 0.2):
#     chat_model = ChatModel.from_pretrained("chat-bison@001")

#     # TODO developer - override these parameters as needed:
#     parameters = {
#         "temperature": temperature,  # Temperature controls the degree of randomness in token selection.
#         "max_output_tokens": 256,  # Token limit determines the maximum amount of text output.
#         "top_p": 0.95,  # Tokens are selected from most probable to least until the sum of their probabilities equals the top_p value.
# #         "top_k": 40,  # A top_k of 1 means the selected token is the most probable among all tokens.
#     }

#     chat = chat_model.start_chat(
#         context=SOPContext,
#         examples=[
#             InputOutputTextPair(
#             ),
#         ],
#     )

#     response = chat.send_message(
#         query, **parameters
#           )
#     print(f"Response from Model: {response.text}")


In [ ]:
# query = "how does deferred Giveaway works for offer"
# query = "how does I create a Promise Callout Offer"
query = "What is insufficient participation ?"
relevantDocs = db.max_marginal_relevance_search(query, k=20)
# relevantDocs = db.similarity_search_with_score(query, k=10)

print(len(relevantDocs))
# print(relevantDocs)

qdocs = "".join([relevantDocs[i].page_content for i in range(len(relevantDocs))])
# print(qdocs)
for i in range(0,len(qdocs)-2):
    print(relevantDocs[i])
    relevantDocs[i].page_content
    print(item, "======\n")

In [ ]:

role = """
You are offer related query answering module for promotions platform on Flipkart.\
You answer questions with a high degree of precision & wherever you do not have enough precision, \
you always provide an output saying the answers might not be fully correct. \
Promotions Platform has capabilities to create and maintain an offer via its \
offer creation console- 'Grumbles' and maintenance console-- 'Casper'. Business users\
can ask multiple questions related to capabilities for creating and maintaining an offer.
"""

prompt = f""" 
Role: {role}
Follow exactly these 3 steps:
1. Read the context below and aggregrate this data
Context : {qdocs}
2. Answer the question using only this context
3. Show the source for your answers
User Question: {query}

If you don't have any context and are unsure of the answer, reply that you don't know about this topic.

"""
print(len(prompt))

In [ ]:
print(getResponse(prompt))

In [ ]:

# response = vertex_llm_text.call_as_llm(f"{qdocs} Question: What is Incentivised offer.") 

# print (sopContext)

# print(prompt)
# question =  f"{qdocs} {prompt}"
# print(question)
# print(getResponse(question))
# display(Markdown(response))

# getChatResponse()

In [ ]:
!gcloud auth application-default login